In [16]:
%reload_ext autoreload
%autoreload 2
%env THEANO_FLAGS=device=gpu4,lib.cnmem=0.5,floatX=float32
%env USER_EMB_SIZE=256
from __future__ import unicode_literals

import sys
sys.path.insert(0, '../')
from mymodule.neural.seq2seq import *
from mymodule.neural.discriminator import DssmModel
from mymodule.base_stuff import Vocab, phrase2matrix
from mymodule.twitter_stuff import *


from agentnet.utils.persistence import save,load
import matplotlib.pyplot as plt
import seaborn as sns
import codecs
import theano
import theano.tensor as T
from functools import partial
from tqdm import tqdm
import pickle

%matplotlib inline

env: THEANO_FLAGS=device=gpu4,lib.cnmem=0.5,floatX=float32
env: USER_EMB_SIZE=256


In [36]:
def load_seq2seq(vocab, weights_path):
    enc = Enc(vocab)
    dec = Dec(vocab, enc)
    gentest = GenTest(vocab, enc, dec)
    gentrain = GenTrain(vocab, enc, dec, gentest)
    
    load(gentest.recurrence, weights_path)
    
    return gentrain,gentest

def load_dssm(vocab):
    dssm_model = DssmModel(vocab, 1000)
    load(dssm_model.l_user_semantic, '../weights/dssm_model_l_user_semantic.pkl')
    load(dssm_model.l_utt_semantic, '../weights/dssm_model_l_utt_semantic.pkl')
    return dssm_model

def calc_perplexity(gentrain, vocab, iterator_fn):
    llhs = []
    for i, (context,answer) in tqdm(enumerate(iterator_fn())):
        if i >= 500:
            break
        x = phrase2matrix(context, vocab, normalize=True)
        y = phrase2matrix(answer, vocab, normalize=True)
        llhs.append(float(gentrain.get_llh(x, y)))
    return llhs

def calc_dssm_score(gentest, dssm_model, user_id, iterator_fn):
    scores = []
    for i, (context, _) in tqdm(enumerate(iterator_fn())):
        if i >= 10000:
            break
        ans = gentest.reply(context[0], normalize=True)
        scores.append(dssm_model.similarity(user_id, ans))
    return np.mean(scores)

In [17]:
vocab = Vocab.read_from_file('../twitter_data/tokens.txt')
dssm_model = load_dssm(vocab)

with open('../twitter_data/selected_uids_dict.pkl', 'rb') as fin:
    uid_map = pickle.load(fin)
    
USER_ID = uid_map['24203097']

In [37]:
general_perplexity_iterator = lambda: iterate_minibatches_twitter(
    '../twitter_data/twitter_valid', vocab, 64, convert_to_matrix=False)
user_perplexity_iterator = lambda: iterate_minibatches_twitter_user_chains(
    '../twitter_data/user_24203097_chains_valid', vocab, 64, convert_to_matrix=False)

dssm_score_iterator = lambda: iterate_minibatches_twitter(
    '../twitter_data/twitter_valid', vocab, 1, convert_to_matrix=False)


# 0. Original seq2seq LM on twitter data

In [5]:
perplexities = {}
dssm_scores = {}

In [6]:
weights_path = '../weights/test_twitter_seq2seq_backup.pkl'

gentrain,gentest = load_seq2seq(vocab, weights_path)

In [14]:
perplexities['0.LM.valid'] = np.exp(np.mean(calc_perplexity(gentrain, vocab, general_perplexity_iterator)))

In [10]:
perplexities['0.LM.valid-user'] = np.exp(np.mean(calc_perplexity(gentrain, vocab, user_perplexity_iterator)))

15it [00:03,  5.05it/s]


In [15]:
perplexities

{u'0.LM.valid': 4.235, u'0.LM.valid-user': 5.2491468439296778}

In [38]:
dssm_scores['0.LM.user_24203097'] = calc_dssm_score(gentest, dssm_model, USER_ID, dssm_score_iterator)


0it [00:00, ?it/s]
4it [00:00, 36.25it/s]
8it [00:00, 36.39it/s]
12it [00:00, 36.98it/s]
16it [00:00, 36.98it/s]
20it [00:00, 36.32it/s]
24it [00:00, 36.01it/s]
28it [00:00, 36.06it/s]
32it [00:00, 35.97it/s]
36it [00:00, 36.41it/s]
40it [00:01, 36.15it/s]
44it [00:01, 36.26it/s]
48it [00:01, 35.68it/s]
52it [00:01, 34.88it/s]
56it [00:01, 34.70it/s]
60it [00:01, 34.56it/s]
64it [00:01, 34.14it/s]
68it [00:01, 33.70it/s]
72it [00:02, 33.60it/s]
76it [00:02, 33.63it/s]
80it [00:02, 32.53it/s]
84it [00:02, 32.24it/s]
88it [00:02, 31.98it/s]
92it [00:02, 31.73it/s]
96it [00:02, 33.04it/s]
100it [00:02, 33.96it/s]
104it [00:03, 35.14it/s]
108it [00:03, 36.08it/s]
112it [00:03, 36.65it/s]
116it [00:03, 36.96it/s]
120it [00:03, 37.03it/s]
124it [00:03, 37.27it/s]
128it [00:03, 37.60it/s]
132it [00:03, 38.04it/s]
136it [00:03, 38.25it/s]
140it [00:03, 37.34it/s]
144it [00:04, 36.90it/s]
148it [00:04, 34.68it/s]
152it [00:04, 34.94it/s]
156it [00:04, 35.04it/s]
160it [00:04, 35.37it/s]
164it 

1168it [00:32, 34.19it/s]
1172it [00:33, 34.90it/s]
1176it [00:33, 35.48it/s]
1180it [00:33, 35.57it/s]
1184it [00:33, 34.49it/s]
1188it [00:33, 34.65it/s]
1192it [00:33, 35.04it/s]
1196it [00:33, 35.29it/s]
1200it [00:33, 34.70it/s]
1204it [00:33, 34.59it/s]
1208it [00:34, 34.08it/s]
1212it [00:34, 33.98it/s]
1216it [00:34, 34.03it/s]
1220it [00:34, 33.33it/s]
1224it [00:34, 33.32it/s]
1228it [00:34, 33.69it/s]
1232it [00:34, 33.89it/s]
1236it [00:34, 33.73it/s]
1240it [00:35, 33.80it/s]
1244it [00:35, 33.92it/s]
1248it [00:35, 33.81it/s]
1252it [00:35, 33.75it/s]
1256it [00:35, 33.98it/s]
1260it [00:35, 34.02it/s]
1264it [00:35, 33.94it/s]
1268it [00:35, 34.09it/s]
1272it [00:35, 34.04it/s]
1276it [00:36, 33.79it/s]
1280it [00:36, 33.29it/s]
1284it [00:36, 32.33it/s]
1288it [00:36, 32.28it/s]
1292it [00:36, 31.87it/s]
1296it [00:36, 31.94it/s]
1300it [00:36, 32.13it/s]
1304it [00:36, 32.19it/s]
1308it [00:37, 31.84it/s]
1312it [00:37, 32.11it/s]
1316it [00:37, 32.56it/s]
1320it [00:3

2296it [01:06, 34.05it/s]
2300it [01:06, 33.68it/s]
2304it [01:06, 34.07it/s]
2308it [01:06, 34.52it/s]
2312it [01:06, 34.82it/s]
2316it [01:06, 34.88it/s]
2320it [01:06, 34.67it/s]
2324it [01:07, 34.38it/s]
2328it [01:07, 34.34it/s]
2332it [01:07, 34.17it/s]
2336it [01:07, 34.50it/s]
2340it [01:07, 34.58it/s]
2344it [01:07, 34.87it/s]
2348it [01:07, 34.25it/s]
2352it [01:07, 34.36it/s]
2356it [01:07, 34.62it/s]
2360it [01:08, 34.16it/s]
2364it [01:08, 34.69it/s]
2368it [01:08, 34.32it/s]
2372it [01:08, 34.12it/s]
2376it [01:08, 34.24it/s]
2380it [01:08, 34.32it/s]
2384it [01:08, 34.19it/s]
2388it [01:08, 34.25it/s]
2392it [01:09, 34.30it/s]
2396it [01:09, 34.52it/s]
2400it [01:09, 34.39it/s]
2404it [01:09, 34.55it/s]
2408it [01:09, 34.43it/s]
2412it [01:09, 33.91it/s]
2416it [01:09, 33.94it/s]
2420it [01:09, 34.61it/s]
2424it [01:09, 35.23it/s]
2428it [01:10, 35.18it/s]
2432it [01:10, 35.34it/s]
2436it [01:10, 35.53it/s]
2440it [01:10, 35.96it/s]
2444it [01:10, 36.37it/s]
2448it [01:1

3424it [01:38, 35.46it/s]
3428it [01:38, 34.74it/s]
3432it [01:38, 34.82it/s]
3436it [01:38, 35.04it/s]
3440it [01:38, 35.81it/s]
3444it [01:38, 36.09it/s]
3448it [01:38, 36.57it/s]
3452it [01:38, 35.34it/s]
3456it [01:39, 35.55it/s]
3460it [01:39, 31.32it/s]
3464it [01:39, 33.20it/s]
3468it [01:39, 33.60it/s]
3472it [01:39, 33.99it/s]
3476it [01:39, 34.26it/s]
3480it [01:39, 34.38it/s]
3484it [01:39, 35.37it/s]
3488it [01:39, 35.88it/s]
3492it [01:40, 36.78it/s]
3496it [01:40, 36.14it/s]
3500it [01:40, 36.37it/s]
3504it [01:40, 36.66it/s]
3508it [01:40, 36.69it/s]
3512it [01:40, 37.07it/s]
3516it [01:40, 36.87it/s]
3520it [01:40, 37.34it/s]
3524it [01:40, 37.43it/s]
3528it [01:41, 37.27it/s]
3532it [01:41, 36.87it/s]
3536it [01:41, 36.74it/s]
3540it [01:41, 36.48it/s]
3544it [01:41, 36.67it/s]
3548it [01:41, 36.67it/s]
3552it [01:41, 37.09it/s]
3556it [01:41, 36.18it/s]
3560it [01:41, 36.13it/s]
3564it [01:42, 36.37it/s]
3568it [01:42, 36.44it/s]
3572it [01:42, 36.46it/s]
3576it [01:4

4552it [02:10, 36.72it/s]
4556it [02:10, 37.10it/s]
4560it [02:11, 37.05it/s]
4564it [02:11, 37.18it/s]
4568it [02:11, 37.16it/s]
4572it [02:11, 37.17it/s]
4576it [02:11, 36.30it/s]
4580it [02:11, 36.00it/s]
4584it [02:11, 35.01it/s]
4588it [02:11, 35.18it/s]
4592it [02:11, 35.17it/s]
4596it [02:12, 35.70it/s]
4600it [02:12, 36.19it/s]
4604it [02:12, 36.42it/s]
4608it [02:12, 36.56it/s]
4612it [02:12, 36.42it/s]
4616it [02:12, 36.26it/s]
4620it [02:12, 35.80it/s]
4624it [02:12, 35.92it/s]
4628it [02:12, 35.27it/s]
4632it [02:13, 34.83it/s]
4636it [02:13, 34.81it/s]
4640it [02:13, 34.82it/s]
4644it [02:13, 33.94it/s]
4648it [02:13, 33.69it/s]
4652it [02:13, 34.41it/s]
4656it [02:13, 35.08it/s]
4660it [02:13, 35.72it/s]
4664it [02:13, 36.36it/s]
4668it [02:14, 36.13it/s]
4672it [02:14, 35.62it/s]
4676it [02:14, 35.38it/s]
4680it [02:14, 35.06it/s]
4684it [02:14, 35.59it/s]
4688it [02:14, 35.43it/s]
4692it [02:14, 35.44it/s]
4696it [02:14, 36.22it/s]
4700it [02:14, 35.87it/s]
4704it [02:1

5680it [02:43, 34.15it/s]
5684it [02:43, 34.03it/s]
5688it [02:43, 33.95it/s]
5692it [02:43, 34.09it/s]
5696it [02:44, 34.21it/s]
5700it [02:44, 34.41it/s]
5704it [02:44, 34.04it/s]
5708it [02:44, 33.81it/s]
5712it [02:44, 34.15it/s]
5716it [02:44, 34.46it/s]
5720it [02:44, 34.35it/s]
5724it [02:44, 33.79it/s]
5728it [02:44, 34.22it/s]
5732it [02:45, 34.40it/s]
5736it [02:45, 34.38it/s]
5740it [02:45, 34.55it/s]
5744it [02:45, 34.46it/s]
5748it [02:45, 34.48it/s]
5752it [02:45, 34.49it/s]
5756it [02:45, 34.49it/s]
5760it [02:45, 34.90it/s]
5764it [02:46, 34.12it/s]
5768it [02:46, 33.48it/s]
5772it [02:46, 33.78it/s]
5776it [02:46, 34.03it/s]
5780it [02:46, 33.06it/s]
5784it [02:46, 31.83it/s]
5788it [02:46, 32.11it/s]
5792it [02:46, 31.75it/s]
5796it [02:47, 31.93it/s]
5800it [02:47, 32.04it/s]
5804it [02:47, 31.56it/s]
5808it [02:47, 31.87it/s]
5812it [02:47, 31.94it/s]
5816it [02:47, 32.57it/s]
5820it [02:47, 33.38it/s]
5824it [02:47, 33.09it/s]
5828it [02:47, 33.20it/s]
5832it [02:4

6782it [03:16, 36.16it/s]
6786it [03:16, 36.03it/s]
6790it [03:16, 35.65it/s]
6794it [03:16, 35.56it/s]
6798it [03:16, 35.57it/s]
6802it [03:16, 33.83it/s]
6806it [03:17, 31.56it/s]
6810it [03:17, 29.80it/s]
6814it [03:17, 28.91it/s]
6817it [03:17, 28.83it/s]
6820it [03:17, 28.67it/s]
6823it [03:17, 28.57it/s]
6826it [03:17, 27.98it/s]
6829it [03:17, 26.81it/s]
6832it [03:18, 26.14it/s]
6835it [03:18, 26.69it/s]
6838it [03:18, 26.72it/s]
6841it [03:18, 26.75it/s]
6844it [03:18, 26.66it/s]
6847it [03:18, 26.88it/s]
6850it [03:18, 27.11it/s]
6853it [03:18, 27.24it/s]
6856it [03:18, 27.79it/s]
6860it [03:19, 28.86it/s]
6864it [03:19, 31.10it/s]
6868it [03:19, 32.60it/s]
6872it [03:19, 33.49it/s]
6876it [03:19, 34.44it/s]
6880it [03:19, 34.92it/s]
6884it [03:19, 34.82it/s]
6888it [03:19, 34.59it/s]
6892it [03:19, 34.98it/s]
6896it [03:20, 35.56it/s]
6900it [03:20, 35.99it/s]
6904it [03:20, 35.99it/s]
6908it [03:20, 35.92it/s]
6912it [03:20, 35.68it/s]
6916it [03:20, 35.49it/s]
6920it [03:2

7896it [03:48, 35.95it/s]
7900it [03:48, 35.08it/s]
7904it [03:48, 35.10it/s]
7908it [03:48, 35.23it/s]
7912it [03:48, 34.97it/s]
7916it [03:48, 34.91it/s]
7920it [03:48, 34.90it/s]
7924it [03:48, 34.61it/s]
7928it [03:49, 34.08it/s]
7932it [03:49, 33.88it/s]
7936it [03:49, 33.78it/s]
7940it [03:49, 34.29it/s]
7944it [03:49, 34.54it/s]
7948it [03:49, 34.22it/s]
7952it [03:49, 34.36it/s]
7956it [03:49, 34.52it/s]
7960it [03:49, 34.49it/s]
7964it [03:50, 34.55it/s]
7968it [03:50, 34.53it/s]
7972it [03:50, 33.90it/s]
7976it [03:50, 34.58it/s]
7980it [03:50, 35.05it/s]
7984it [03:50, 35.39it/s]
7988it [03:50, 35.87it/s]
7992it [03:50, 35.90it/s]
7996it [03:50, 36.19it/s]
8000it [03:51, 35.99it/s]
8004it [03:51, 36.06it/s]
8008it [03:51, 35.51it/s]
8012it [03:51, 34.97it/s]
8016it [03:51, 35.13it/s]
8020it [03:51, 36.16it/s]
8024it [03:51, 36.00it/s]
8028it [03:51, 35.86it/s]
8032it [03:51, 35.93it/s]
8036it [03:52, 35.82it/s]
8040it [03:52, 36.41it/s]
8044it [03:52, 36.76it/s]
8048it [03:5

9024it [04:20, 34.88it/s]
9028it [04:20, 34.56it/s]
9032it [04:20, 34.32it/s]
9036it [04:21, 33.70it/s]
9040it [04:21, 32.54it/s]
9044it [04:21, 33.16it/s]
9048it [04:21, 33.57it/s]
9052it [04:21, 33.81it/s]
9056it [04:21, 34.32it/s]
9060it [04:21, 33.21it/s]
9064it [04:21, 33.69it/s]
9068it [04:22, 34.13it/s]
9072it [04:22, 33.79it/s]
9076it [04:22, 34.04it/s]
9080it [04:22, 33.34it/s]
9084it [04:22, 33.51it/s]
9088it [04:22, 33.66it/s]
9092it [04:22, 34.20it/s]
9096it [04:22, 34.83it/s]
9100it [04:22, 35.09it/s]
9104it [04:23, 34.47it/s]
9108it [04:23, 34.43it/s]
9112it [04:23, 34.46it/s]
9116it [04:23, 34.23it/s]
9120it [04:23, 34.31it/s]
9124it [04:23, 34.55it/s]
9128it [04:23, 34.24it/s]
9132it [04:23, 34.61it/s]
9136it [04:23, 34.09it/s]
9140it [04:24, 34.24it/s]
9144it [04:24, 34.31it/s]
9148it [04:24, 34.67it/s]
9152it [04:24, 34.81it/s]
9156it [04:24, 34.17it/s]
9160it [04:24, 34.06it/s]
9164it [04:24, 34.28it/s]
9168it [04:24, 34.33it/s]
9172it [04:25, 33.40it/s]
9176it [04:2

In [39]:
dssm_scores

{u'0.LM.user_24203097': 0.25867391}

# 1. LM finetuned on uid `24203097` msgs

In [40]:
weights_path = '../weights/test_twitter_seq2seq_finetuned_by_llh_on_user.pkl'
gentrain,gentest = load_seq2seq(vocab, weights_path)

perplexities['1.LM-tuned.valid'] = np.exp(np.mean(calc_perplexity(gentrain, vocab, general_perplexity_iterator)))
perplexities['1.LM-tuned.valid-user'] = np.exp(np.mean(calc_perplexity(gentrain, vocab, user_perplexity_iterator)))
dssm_scores['1.LM-tuned.user_24203097'] = calc_dssm_score(gentest, dssm_model, USER_ID, dssm_score_iterator)



0it [00:00, ?it/s]

1it [00:00,  2.05it/s]

2it [00:01,  1.87it/s]

3it [00:01,  2.20it/s]

4it [00:01,  2.59it/s]

5it [00:01,  2.87it/s]

6it [00:02,  3.06it/s]

7it [00:02,  2.40it/s]

8it [00:03,  1.85it/s]


9it [00:04,  1.89it/s]

10it [00:04,  1.97it/s]

11it [00:04,  2.29it/s]

12it [00:05,  2.69it/s]

13it [00:05,  2.70it/s]

14it [00:05,  3.11it/s]

15it [00:05,  3.17it/s]

16it [00:06,  3.40it/s]

17it [00:06,  3.76it/s]

18it [00:06,  3.83it/s]

19it [00:06,  3.74it/s]

20it [00:07,  3.78it/s]

21it [00:07,  3.96it/s]

22it [00:07,  4.11it/s]

23it [00:07,  3.76it/s]

24it [00:08,  3.86it/s]

25it [00:08,  3.89it/s]

26it [00:08,  3.14it/s]

27it [00:09,  3.46it/s]

28it [00:09,  3.62it/s]

29it [00:09,  3.88it/s]

30it [00:09,  3.91it/s]

31it [00:10,  3.97it/s]

32it [00:14,  1.52s/it]

33it [00:14,  1.14s/it]

34it [00:15,  1.14it/s]

35it [00:15,  1.49it/s]

36it [00:15,  1.84it/s]

37it [00:15,  2.23it/s]

38it [00:15,  2.62it/s]

39it [00:16,  2.93it/s]

40it [00:16

259it [01:09,  4.21it/s]

260it [01:09,  4.20it/s]

261it [01:10,  4.17it/s]

262it [01:10,  4.37it/s]

263it [01:10,  4.31it/s]

264it [01:10,  4.47it/s]

265it [01:11,  4.39it/s]

266it [01:11,  4.47it/s]

267it [01:11,  4.52it/s]

268it [01:11,  4.23it/s]

269it [01:20,  2.63s/it]

270it [01:20,  1.91s/it]

271it [01:20,  1.41s/it]

272it [01:20,  1.06s/it]

273it [01:21,  1.21it/s]

274it [01:21,  1.55it/s]

275it [01:21,  1.88it/s]

276it [01:21,  2.26it/s]

277it [01:22,  2.42it/s]

278it [01:22,  2.83it/s]

279it [01:22,  3.19it/s]

280it [01:22,  3.42it/s]

281it [01:23,  3.72it/s]

282it [01:23,  3.83it/s]

283it [01:23,  3.95it/s]

284it [01:23,  4.03it/s]

285it [01:23,  4.08it/s]

286it [01:24,  4.17it/s]

287it [01:24,  4.16it/s]

288it [01:24,  4.15it/s]

289it [01:24,  4.25it/s]

290it [01:25,  4.13it/s]

291it [01:25,  4.13it/s]

292it [01:25,  4.02it/s]

293it [01:25,  4.07it/s]

294it [01:26,  4.24it/s]

295it [01:26,  4.31it/s]

296it [01:26,  4.25it/s]

297it [01:26

13it [00:02,  5.37it/s]


14it [00:02,  5.14it/s]


15it [00:02,  5.29it/s]





0it [00:00, ?it/s]


1it [00:00,  8.26it/s]


5it [00:00, 10.73it/s]


9it [00:00, 13.59it/s]


13it [00:00, 16.72it/s]


17it [00:00, 19.94it/s]


21it [00:00, 22.99it/s]


25it [00:00, 25.78it/s]


29it [00:00, 27.73it/s]


33it [00:01, 29.60it/s]


37it [00:01, 31.44it/s]


41it [00:01, 32.30it/s]


45it [00:01, 33.29it/s]


49it [00:01, 34.16it/s]


53it [00:01, 34.71it/s]


57it [00:01, 34.68it/s]


61it [00:01, 35.02it/s]


65it [00:01, 34.69it/s]


69it [00:02, 34.95it/s]


73it [00:02, 35.04it/s]


77it [00:02, 34.55it/s]


81it [00:02, 33.56it/s]


85it [00:02, 33.22it/s]


89it [00:02, 33.38it/s]


93it [00:02, 33.75it/s]


97it [00:02, 33.84it/s]


101it [00:02, 34.34it/s]


105it [00:03, 34.89it/s]


109it [00:03, 35.20it/s]


113it [00:03, 35.11it/s]


117it [00:03, 35.21it/s]


121it [00:03, 35.13it/s]


125it [00:03, 35.05it/s]


129it [00:03, 35.77it/s]


133it [00:03, 36.23it/s]


137it [0

897it [00:24, 37.05it/s]


901it [00:24, 37.28it/s]


905it [00:24, 37.37it/s]


909it [00:25, 37.07it/s]


913it [00:25, 37.01it/s]


917it [00:25, 37.04it/s]


921it [00:25, 36.82it/s]


925it [00:25, 36.43it/s]


929it [00:25, 36.59it/s]


933it [00:25, 36.95it/s]


937it [00:25, 36.92it/s]


941it [00:25, 37.18it/s]


945it [00:26, 36.85it/s]


949it [00:26, 36.84it/s]


953it [00:26, 36.93it/s]


957it [00:26, 36.46it/s]


961it [00:26, 36.22it/s]


965it [00:26, 35.88it/s]


969it [00:26, 36.17it/s]


973it [00:26, 36.59it/s]


977it [00:26, 36.64it/s]


981it [00:27, 35.37it/s]


985it [00:27, 35.71it/s]


989it [00:27, 36.21it/s]


993it [00:27, 36.05it/s]


997it [00:27, 35.83it/s]


1001it [00:27, 36.25it/s]


1005it [00:27, 36.35it/s]


1009it [00:27, 35.49it/s]


1013it [00:27, 35.78it/s]


1017it [00:28, 36.25it/s]


1021it [00:28, 36.62it/s]


1025it [00:28, 36.18it/s]


1029it [00:28, 36.01it/s]


1033it [00:28, 36.50it/s]


1037it [00:28, 36.92it/s]


1041it [00:28, 36.

1785it [00:49, 34.00it/s]


1789it [00:50, 34.04it/s]


1793it [00:50, 33.83it/s]


1797it [00:50, 34.00it/s]


1801it [00:50, 34.47it/s]


1805it [00:50, 34.42it/s]


1809it [00:50, 34.59it/s]


1813it [00:50, 34.49it/s]


1817it [00:50, 34.29it/s]


1821it [00:51, 34.26it/s]


1825it [00:51, 34.46it/s]


1829it [00:51, 34.65it/s]


1833it [00:51, 34.85it/s]


1837it [00:51, 34.81it/s]


1841it [00:51, 35.30it/s]


1845it [00:51, 35.14it/s]


1849it [00:51, 35.00it/s]


1853it [00:51, 35.25it/s]


1857it [00:52, 34.74it/s]


1861it [00:52, 34.73it/s]


1865it [00:52, 34.85it/s]


1869it [00:52, 34.69it/s]


1873it [00:52, 34.06it/s]


1877it [00:52, 34.37it/s]


1881it [00:52, 34.63it/s]


1885it [00:52, 34.50it/s]


1889it [00:53, 33.76it/s]


1893it [00:53, 33.59it/s]


1897it [00:53, 33.61it/s]


1901it [00:53, 33.97it/s]


1905it [00:53, 33.73it/s]


1909it [00:53, 33.84it/s]


1913it [00:53, 34.34it/s]


1917it [00:53, 34.42it/s]


1921it [00:53, 34.01it/s]


1925it [00:54, 34.55

...

6205it [02:55, 31.38it/s]


6209it [02:55, 32.13it/s]


6213it [02:55, 32.60it/s]


6217it [02:55, 33.34it/s]


6221it [02:55, 33.39it/s]


6225it [02:55, 34.13it/s]


6229it [02:55, 34.15it/s]


6233it [02:56, 34.39it/s]


6237it [02:56, 34.36it/s]


6241it [02:56, 33.90it/s]


6245it [02:56, 34.05it/s]


6249it [02:56, 34.37it/s]


6253it [02:56, 34.33it/s]


6257it [02:56, 34.58it/s]


6261it [02:56, 34.88it/s]


6265it [02:56, 35.07it/s]


6269it [02:57, 35.05it/s]


6273it [02:57, 34.46it/s]


6277it [02:57, 33.98it/s]


6281it [02:57, 34.47it/s]


6285it [02:57, 33.46it/s]


6289it [02:57, 33.81it/s]


6293it [02:57, 34.07it/s]


6297it [02:57, 34.53it/s]


6301it [02:58, 34.82it/s]


6305it [02:58, 34.82it/s]


6309it [02:58, 34.61it/s]


6313it [02:58, 34.46it/s]


6317it [02:58, 34.13it/s]


6321it [02:58, 33.48it/s]


6325it [02:58, 33.69it/s]


6329it [02:58, 33.33it/s]


6333it [02:58, 33.65it/s]


6337it [02:59, 33.82it/s]


6341it [02:59, 33.80it/s]


6345it [02:59, 32.98

7089it [03:20, 34.19it/s]


7093it [03:20, 33.86it/s]


7097it [03:21, 33.14it/s]


7101it [03:21, 33.66it/s]


7105it [03:21, 34.02it/s]


7109it [03:21, 33.92it/s]


7113it [03:21, 34.10it/s]


7117it [03:21, 34.40it/s]


7121it [03:21, 34.26it/s]


7125it [03:21, 34.24it/s]


7129it [03:21, 33.95it/s]


7133it [03:22, 33.91it/s]


7137it [03:22, 34.07it/s]


7141it [03:22, 33.65it/s]


7145it [03:22, 33.87it/s]


7149it [03:22, 33.33it/s]


7153it [03:22, 33.21it/s]


7157it [03:22, 33.42it/s]


7161it [03:22, 33.57it/s]


7165it [03:23, 33.40it/s]


7169it [03:23, 33.80it/s]


7173it [03:23, 33.89it/s]


7177it [03:23, 34.31it/s]


7181it [03:23, 34.65it/s]


7185it [03:23, 35.05it/s]


7189it [03:23, 35.33it/s]


7193it [03:23, 35.44it/s]


7197it [03:23, 34.25it/s]


7201it [03:24, 34.55it/s]


7205it [03:24, 34.49it/s]


7209it [03:24, 34.83it/s]


7213it [03:24, 34.48it/s]


7217it [03:24, 34.67it/s]


7221it [03:24, 34.34it/s]


7225it [03:24, 33.92it/s]


7229it [03:24, 34.03

7972it [03:47, 32.39it/s]


7976it [03:47, 32.76it/s]


7980it [03:47, 32.98it/s]


7984it [03:47, 32.98it/s]


7988it [03:47, 33.16it/s]


7992it [03:47, 33.02it/s]


7996it [03:47, 33.33it/s]


8000it [03:47, 33.81it/s]


8004it [03:47, 34.08it/s]


8008it [03:48, 32.90it/s]


8012it [03:48, 32.16it/s]


8016it [03:48, 32.11it/s]


8020it [03:48, 32.94it/s]


8024it [03:48, 32.79it/s]


8028it [03:48, 32.87it/s]


8032it [03:48, 32.96it/s]


8036it [03:48, 33.08it/s]


8040it [03:49, 33.48it/s]


8044it [03:49, 33.71it/s]


8048it [03:49, 33.50it/s]


8052it [03:49, 32.98it/s]


8056it [03:49, 33.53it/s]


8060it [03:49, 33.91it/s]


8064it [03:49, 33.89it/s]


8068it [03:49, 34.04it/s]


8072it [03:50, 34.10it/s]


8076it [03:50, 33.77it/s]


8080it [03:50, 33.52it/s]


8084it [03:50, 33.53it/s]


8088it [03:50, 33.66it/s]


8092it [03:50, 33.48it/s]


8096it [03:50, 32.85it/s]


8100it [03:50, 33.04it/s]


8104it [03:50, 32.63it/s]


8108it [03:51, 32.61it/s]


8112it [03:51, 32.58

8856it [04:13, 33.74it/s]


8860it [04:13, 33.24it/s]


8864it [04:13, 33.70it/s]


8868it [04:13, 33.62it/s]


8872it [04:13, 34.14it/s]


8876it [04:13, 33.97it/s]


8880it [04:13, 33.50it/s]


8884it [04:13, 34.03it/s]


8888it [04:14, 35.40it/s]


8892it [04:14, 35.93it/s]


8896it [04:14, 36.53it/s]


8900it [04:14, 36.10it/s]


8904it [04:14, 35.37it/s]


8908it [04:14, 34.82it/s]


8912it [04:14, 34.75it/s]


8916it [04:14, 34.63it/s]


8920it [04:15, 34.64it/s]


8924it [04:15, 34.99it/s]


8928it [04:15, 34.63it/s]


8932it [04:15, 33.77it/s]


8936it [04:15, 34.43it/s]


8940it [04:15, 34.33it/s]


8944it [04:15, 34.06it/s]


8948it [04:15, 34.63it/s]


8952it [04:15, 35.29it/s]


8956it [04:16, 35.00it/s]


8960it [04:16, 34.64it/s]


8964it [04:16, 33.29it/s]


8968it [04:16, 33.67it/s]


8972it [04:16, 34.09it/s]


8976it [04:16, 33.43it/s]


8980it [04:16, 33.32it/s]


8984it [04:16, 33.32it/s]


8988it [04:16, 34.01it/s]


8992it [04:17, 33.84it/s]


8996it [04:17, 33.50

9740it [04:38, 36.32it/s]


9744it [04:38, 36.43it/s]


9748it [04:38, 35.55it/s]


9752it [04:38, 35.77it/s]


9756it [04:38, 36.33it/s]


9760it [04:38, 36.30it/s]


9764it [04:39, 35.95it/s]


9768it [04:39, 35.83it/s]


9772it [04:39, 35.64it/s]


9776it [04:39, 36.10it/s]


9780it [04:39, 36.53it/s]


9784it [04:39, 35.65it/s]


9788it [04:39, 36.22it/s]


9792it [04:39, 35.79it/s]


9796it [04:39, 35.60it/s]


9800it [04:40, 35.91it/s]


9804it [04:40, 35.81it/s]


9808it [04:40, 36.30it/s]


9812it [04:40, 36.25it/s]


9816it [04:40, 36.32it/s]


9820it [04:40, 36.58it/s]


9824it [04:40, 35.96it/s]


9828it [04:40, 36.16it/s]


9832it [04:40, 35.99it/s]


9836it [04:41, 35.61it/s]


9840it [04:41, 35.35it/s]


9844it [04:41, 35.57it/s]


9848it [04:41, 35.19it/s]


9852it [04:41, 34.36it/s]


9856it [04:41, 33.78it/s]


9860it [04:41, 34.50it/s]


9864it [04:41, 34.91it/s]


9868it [04:42, 34.52it/s]


9872it [04:42, 35.00it/s]


9876it [04:42, 35.34it/s]


9880it [04:42, 35.84

In [42]:
perplexities

{u'0.LM.valid': 4.235,
 u'0.LM.valid-user': 5.2491468439296778,
 u'1.LM-tuned.valid': 5.7921216066786938,
 u'1.LM-tuned.valid-user': 6.5409135835948815}

In [41]:
dssm_scores

{u'0.LM.user_24203097': 0.25867391, u'1.LM-tuned.user_24203097': 0.38989127}

# 2. LM finetuned on whole twitter but with DSSM weights for uid `24203097`

# 3. LM finetuned with A2C, using DSSM scores as rewards